In [25]:
from pyqpanda import *
import numpy as np
from math import sqrt
import scipy as sp
%matplotlib inline
ket0 = np.array([[1],[0]],dtype=complex)
ket1 = np.array([[0],[1]],dtype=complex)

def question1(quantum_state_vector,qlist):
    cir = pq.QCircuit()
    # create a circuit that can create the input state by $U|0\rangle = |\psi\rangle$, and only use basic gates.
    return cir


def question2(unitary_matrix,qlist):
    cir = pq.QCircuit()
    # create a circuit which matrix form is same to the input matrix, and only use basic gates
    cir << pq.I(qlist)
    return cir

In [2]:
# 一层ansatz线路
def oneCircuit(qlist, beta, gamma):
    vqc = VariationalQuantumCircuit()
    for (u, v) in E:
        vqc.insert(VariationalQuantumGate_CNOT(qlist[u], qlist[v]))
        vqc.insert(VariationalQuantumGate_RZ(qlist[v], tmp_dict["Z{} Z{}".format(u, v)] * 2 * gamma))
        vqc.insert(VariationalQuantumGate_CNOT(qlist[u], qlist[v]))
    for v in V:
        vqc.insert(VariationalQuantumGate_RZ(qlist[v], tmp_dict["Z{}".format(v)] * 2 * gamma))

    for q in qlist:
        vqc.insert(VariationalQuantumGate_RX(q, 2 * beta))
    return vqc


if __name__ == "__main__":
    qubit_num = 3
    psi_target = np.zeros((2**qubit_num,1),dtype=complex)
    psi_target[1][0] = psi_target[2][0] = psi_target[4][0] = 1
    psi_target /= sqrt(3)
    qvm = CPUQVM()
    qvm.init_qvm()
    qubits = qvm.qAlloc_many(qubit_num)

    # 初始化待优化参数
    paras = var(1/2*np.ones((qubit_num*2, 4), dtype='float64'), True)
    
    # 搭建可变量子线路（含参）
    vqc = VariationalQuantumCircuit()
    
    vqc.insert(VQG_U4_batch(qubits[0],paras[0][0],paras[0][1],paras[0][2],paras[0][3]) << CNOT(qubits[0],qubits[1]) << U4(qubits[0],paras[1][0],paras[1][1],paras[1][2],paras[1][3])\
    << U4(qubits[1],paras[2][0],paras[2][1],paras[2][2],paras[2][3]) << CNOT(qubits[1],qubits[2]) << U4(qubits[1],paras[3][0],paras[3][1],paras[3][2],paras[3][3])\
    << U4(qubits[2],paras[4][0],paras[4][1],paras[4][2],paras[4][3]) << CNOT(qubits[2],qubits[0]) << U4(qubits[2],paras[5][0],paras[5][1],paras[5][2],paras[5][3])\


    # 计算vertex cover中的个数（哈密顿量的期望）
    loss = 
    
    # 使用梯度下降法最小化损失函数
    optimizer = MomentumOptimizer.minimize(loss, 1e-3, 0.9)
    leaves = optimizer.get_variables()    # 获得所有可训练的参数

    # 训练迭代100次
    for i in range(100):
        optimizer.run(leaves, 0)
        num = optimizer.get_loss()
        if i % 10 == 0:
            print("train step: ", i, " num:", num)

    # 输出优化后的参数
    print('beta:', eval(beta).reshape(-1))
    print('gamma:', eval(gamma).reshape(-1))

    # 验证结果
    prog = QProg()
    qcir = vqc.feed()
    prog.insert(qcir)
    result = qvm.prob_run_dict(prog, qubits, 2)
    print(result)

NameError: name 'E' is not defined

In [51]:
import torch as tc
import matplotlib.pylab as plt
from torch.optim import Adam

lr, it_time = 1e-3, 50  # 初始学习率与迭代总次数

print('随机生成目标态')
num_qubits = 3
psi_target = tc.randn((2**num_qubits, ), dtype=tc.complex128)
psi_target /= psi_target.norm()

print('初始化变分参数并建立优化器')
paras = tc.randn((num_qubits*2, 4), dtype=tc.float64,
                 requires_grad=True)
optimizer = Adam([paras], lr=lr)

print('建立非参数门')
loss_rec = tc.zeros(it_time, )

print('开始优化')
qvm = CPUQVM()
qvm.init_qvm()
qubits = qvm.qAlloc_many(3)
for t in range(it_time):
    
    prog = QProg()

    prog << U4(qubits[0],paras[0][0],paras[0][1],paras[0][2],paras[0][3]) << CNOT(qubits[0],qubits[1]) << U4(qubits[0],paras[1][0],paras[1][1],paras[1][2],paras[1][3])\
        << U4(qubits[1],paras[2][0],paras[2][1],paras[2][2],paras[2][3]) << CNOT(qubits[1],qubits[2]) << U4(qubits[1],paras[3][0],paras[3][1],paras[3][2],paras[3][3])\
        << U4(qubits[2],paras[4][0],paras[4][1],paras[4][2],paras[4][3]) << CNOT(qubits[2],qubits[0]) << U4(qubits[2],paras[5][0],paras[5][1],paras[5][2],paras[5][3])\
    
    result = qvm.prob_run_dict(prog, qubits, -1)

    psi = np.array(qvm.get_qstate()).reshape(8,1)

    # 计算loss并作反向传播与梯度优化
    loss = 1 - np.linalg.norm(psi.T.conjugate().dot(psi_target))
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    loss_rec[t] = loss.item()
    if t % 20 == 19:
        print('第%d次迭代后，loss = %g' % (t+1, loss.item()))

qvm.finalize()
plt.plot(loss_rec)
plt.xlabel('iteration time')
plt.ylabel('loss')
plt.show()

随机生成目标态
初始化变分参数并建立优化器
建立非参数门
开始优化


AttributeError: 'numpy.float64' object has no attribute 'backward'

In [11]:
# 导入包
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import math
from math import pi
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit.tools.visualization import circuit_drawer
from qiskit.quantum_info import state_fidelity
from qiskit import BasicAer
# 实现一个单元格有多个输出，不要忘记！！！
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
np.set_printoptions(precision=8)
np.set_printoptions(suppress=True)
 
 
 
# Initializing a three-qubit quantum state
desired_vector = [
    1 / math.sqrt(16) * complex(0, 1),
    1 / math.sqrt(8) * complex(1, 0),
    1 / math.sqrt(16) * complex(1, 1),
    0,
    0,
    1 / math.sqrt(8) * complex(1, 2),
    1 / math.sqrt(16) * complex(1, 0),
    0]
 
# Create quantumcircuit
qc = QuantumCircuit(3)
    
#量子态初始化，将第1,2,3个qubit转换为我们想要的叠加态desired_vector
qc.initialize(desired_vector, [0,1,2])

# 指定模拟器statevector_simulator
backend = BasicAer.get_backend('statevector_simulator')

# 执行线路
job = execute(qc,backend)

#获取状态向量，get_statevector和statevector_simulator是对应的
qc_state = job.result().get_statevector(qc)
print(qc_state)
qc.draw()
#计算保真度
state_fidelity(desired_vector,qc_state)

[0.        +0.25j       0.35355339+0.j         0.25      +0.25j
 0.        +0.j         0.        +0.j         0.35355339+0.70710678j
 0.25      -0.j         0.        +0.j        ]


┌───────────────────────────────────────────────────────────────────┐
q_0: ┤0                                                                  ├
     │                                                                   │
q_1: ┤1 Initialize(0.25j,0.35355,0.25+0.25j,0,0,0.35355+0.70711j,0.25,0) ├
     │                                                                   │
q_2: ┤2                                                                  ├
     └───────────────────────────────────────────────────────────────────┘

1.0000000000000004

In [13]:
type(qc)

qiskit.circuit.quantumcircuit.QuantumCircuit